In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import re
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
all_verbs = pd.read_csv('./all_verbs_LR_f.csv').reset_index(drop = True)
all_verbs

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion,Stress Number,First Character,Last Character,Last Character Number,Vowels Quantity
0,0,present,indicative,imperfective,non-causative,negative,1s,passive,intrans,a-me-*ne*-ger-i-em(ɛ),amneˈgerijemɛ,e,0.307692,0,ɛ,15,6
1,1,present,indicative,imperfective,non-causative,negative,2s,passive,intrans,a-me-*ne*-ger-i-i,amneˈgerii,e,0.400000,0,i,5,5
2,2,present,indicative,imperfective,non-causative,negative,3sm,passive,intrans,a-me-*ne*-ger-i-e,amneˈgerije,e,0.363636,0,e,3,5
3,3,present,indicative,imperfective,non-causative,negative,3sf,passive,intrans,a-me-*ne*-ger-i-iɛ,amneˈgeriijɛ,e,0.333333,0,ɛ,15,6
4,4,present,indicative,imperfective,non-causative,negative,1p,passive,intrans,a-me-*ne*-ger-i-om(ɛ),amneˈgerijomɛ,e,0.307692,0,ɛ,15,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44107,44107,past,subjunctive,pluperfect,non-causative,affirmative,3sm,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ,0.545455,0,ɛ,15,5
44108,44108,past,subjunctive,pluperfect,non-causative,affirmative,3sf,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ,0.545455,0,ɛ,15,5
44109,44109,past,subjunctive,pluperfect,non-causative,affirmative,1p,active,trans,ʌ-χʌst-*abimon*,ʌχʌstɛˈbimon,ɛ,0.500000,0,n,8,5
44110,44110,past,subjunctive,pluperfect,non-causative,affirmative,2p,active,trans,ʌ-χʌst-*abion*,ʌχʌstɛˈbijon,ɛ,0.500000,0,n,8,5


In [5]:
from sklearn import tree
def decision_tree_models():
    features = ['Tense', 'Modality', 'Aspect', 'Causation', 'Polarity', 'Person', 'Voice', 'Transitivity']
    model_info = {}
    for feature in features:

        model = tree.DecisionTreeClassifier() 
        X = all_verbs['Generated Verb-SR'] 
        y = all_verbs[feature] 
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 1)
        count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3))
        X_train = count_vect.fit_transform(X_train)
        X_test = count_vect.transform(X_test) 
        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        y_test = le.transform(y_test)
        print(le.classes_)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        matrix = confusion_matrix(y_test, y_pred)
        word_predict = model.predict(count_vect.transform(np.array(['ʌbɛmdinijɛst*ɛ*vijɛ'])))
        accuracy = accuracy_score(y_test, y_pred)
        model_info[feature] = { 'model': model,'accuracy': accuracy,'word prediction': word_predict, 'confusion matrix': matrix }
    
    return model_info

In [6]:
models = decision_tree_models()

['past' 'present']
['imperative' 'indicative' 'subjunctive']
['imperfective' 'neutral' 'perfect' 'pluperfect']
['causative' 'non-causative']
['affirmative' 'negative']
['1p' '1s' '2p' '2s' '3p' '3sf' '3sm']
['active' 'passive']
['intrans' 'trans']


In [7]:
models

{'Tense': {'model': DecisionTreeClassifier(),
  'accuracy': 0.998969567905475,
  'word prediction': array([0]),
  'confusion matrix': array([[11112,     4],
         [   11,  3430]])},
 'Modality': {'model': DecisionTreeClassifier(),
  'accuracy': 0.8029813835268256,
  'word prediction': array([1]),
  'confusion matrix': array([[ 175,    2,   73],
         [   7, 6767, 1119],
         [  83, 1584, 4747]])},
 'Aspect': {'model': DecisionTreeClassifier(),
  'accuracy': 0.7989970460946624,
  'word prediction': array([2]),
  'confusion matrix': array([[3151,   26,    0,    0],
         [  34, 4971,   85,    0],
         [   0,  175, 1785, 1181],
         [   0,    0, 1425, 1724]])},
 'Causation': {'model': DecisionTreeClassifier(),
  'accuracy': 0.9998626090540633,
  'word prediction': array([0]),
  'confusion matrix': array([[ 4486,     2],
         [    0, 10069]])},
 'Polarity': {'model': DecisionTreeClassifier(),
  'accuracy': 0.9983513086487601,
  'word prediction': array([0]),
  'con

In [8]:
from sklearn.ensemble import RandomForestClassifier


def random_forest_models():
    features = ['Tense', 'Modality', 'Aspect', 'Causation', 'Polarity', 'Person', 'Voice', 'Transitivity']
    model_info = {}
    for feature in features:


        model = RandomForestClassifier() 
        X = all_verbs['Generated Verb-SR'] 
        y = all_verbs[feature] 
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 1)
        count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3)) 
        X_train = count_vect.fit_transform(X_train)
        X_test = count_vect.transform(X_test) 
        le = LabelEncoder()
        y_train = le.fit_transform(y_train) 
        y_test = le.transform(y_test)
        print(le.classes_)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        matrix = confusion_matrix(y_test, y_pred)
        word_predict = model.predict(count_vect.transform(np.array(['ʌbɛmdinijɛst*ɛ*vijɛ'])))
        accuracy = accuracy_score(y_test, y_pred)
        model_info[feature] = { 'model': model,'accuracy': accuracy,'word prediction': word_predict, 'confusion matrix': matrix }
        
    return model_info


In [9]:
models = random_forest_models()

['past' 'present']
['imperative' 'indicative' 'subjunctive']
['imperfective' 'neutral' 'perfect' 'pluperfect']
['causative' 'non-causative']
['affirmative' 'negative']
['1p' '1s' '2p' '2s' '3p' '3sf' '3sm']
['active' 'passive']
['intrans' 'trans']


In [10]:
models

{'Tense': {'model': RandomForestClassifier(),
  'accuracy': 0.999793913581095,
  'word prediction': array([0]),
  'confusion matrix': array([[11115,     1],
         [    2,  3439]])},
 'Modality': {'model': RandomForestClassifier(),
  'accuracy': 0.8029126880538573,
  'word prediction': array([1]),
  'confusion matrix': array([[ 143,    1,  106],
         [   0, 6704, 1189],
         [  76, 1497, 4841]])},
 'Aspect': {'model': RandomForestClassifier(),
  'accuracy': 0.7960431407570241,
  'word prediction': array([2]),
  'confusion matrix': array([[3138,   39,    0,    0],
         [   8, 4948,  134,    0],
         [   0,  182, 1631, 1328],
         [   0,    0, 1278, 1871]])},
 'Causation': {'model': RandomForestClassifier(),
  'accuracy': 0.9999313045270317,
  'word prediction': array([0]),
  'confusion matrix': array([[ 4487,     1],
         [    0, 10069]])},
 'Polarity': {'model': RandomForestClassifier(),
  'accuracy': 0.9991069588514117,
  'word prediction': array([0]),
  'con

In [11]:
all_verbs = all_verbs.rename({'starred': 'Stressed Vowel'})
all_verbs

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion,Stress Number,First Character,Last Character,Last Character Number,Vowels Quantity
0,0,present,indicative,imperfective,non-causative,negative,1s,passive,intrans,a-me-*ne*-ger-i-em(ɛ),amneˈgerijemɛ,e,0.307692,0,ɛ,15,6
1,1,present,indicative,imperfective,non-causative,negative,2s,passive,intrans,a-me-*ne*-ger-i-i,amneˈgerii,e,0.400000,0,i,5,5
2,2,present,indicative,imperfective,non-causative,negative,3sm,passive,intrans,a-me-*ne*-ger-i-e,amneˈgerije,e,0.363636,0,e,3,5
3,3,present,indicative,imperfective,non-causative,negative,3sf,passive,intrans,a-me-*ne*-ger-i-iɛ,amneˈgeriijɛ,e,0.333333,0,ɛ,15,6
4,4,present,indicative,imperfective,non-causative,negative,1p,passive,intrans,a-me-*ne*-ger-i-om(ɛ),amneˈgerijomɛ,e,0.307692,0,ɛ,15,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44107,44107,past,subjunctive,pluperfect,non-causative,affirmative,3sm,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ,0.545455,0,ɛ,15,5
44108,44108,past,subjunctive,pluperfect,non-causative,affirmative,3sf,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ,0.545455,0,ɛ,15,5
44109,44109,past,subjunctive,pluperfect,non-causative,affirmative,1p,active,trans,ʌ-χʌst-*abimon*,ʌχʌstɛˈbimon,ɛ,0.500000,0,n,8,5
44110,44110,past,subjunctive,pluperfect,non-causative,affirmative,2p,active,trans,ʌ-χʌst-*abion*,ʌχʌstɛˈbijon,ɛ,0.500000,0,n,8,5


In [12]:
all_verbs['Stress Number'] = all_verbs['Generated Verb-SR'].apply(lambda word: word.rindex('ˈ') / len(word)) #before : goes input variable

In [13]:
all_verbs

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion,Stress Number,First Character,Last Character,Last Character Number,Vowels Quantity
0,0,present,indicative,imperfective,non-causative,negative,1s,passive,intrans,a-me-*ne*-ger-i-em(ɛ),amneˈgerijemɛ,e,0.307692,0,ɛ,15,6
1,1,present,indicative,imperfective,non-causative,negative,2s,passive,intrans,a-me-*ne*-ger-i-i,amneˈgerii,e,0.400000,0,i,5,5
2,2,present,indicative,imperfective,non-causative,negative,3sm,passive,intrans,a-me-*ne*-ger-i-e,amneˈgerije,e,0.363636,0,e,3,5
3,3,present,indicative,imperfective,non-causative,negative,3sf,passive,intrans,a-me-*ne*-ger-i-iɛ,amneˈgeriijɛ,e,0.333333,0,ɛ,15,6
4,4,present,indicative,imperfective,non-causative,negative,1p,passive,intrans,a-me-*ne*-ger-i-om(ɛ),amneˈgerijomɛ,e,0.307692,0,ɛ,15,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44107,44107,past,subjunctive,pluperfect,non-causative,affirmative,3sm,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ,0.545455,0,ɛ,15,5
44108,44108,past,subjunctive,pluperfect,non-causative,affirmative,3sf,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ,0.545455,0,ɛ,15,5
44109,44109,past,subjunctive,pluperfect,non-causative,affirmative,1p,active,trans,ʌ-χʌst-*abimon*,ʌχʌstɛˈbimon,ɛ,0.500000,0,n,8,5
44110,44110,past,subjunctive,pluperfect,non-causative,affirmative,2p,active,trans,ʌ-χʌst-*abion*,ʌχʌstɛˈbijon,ɛ,0.500000,0,n,8,5


In [16]:
all_verbs.to_csv('./all_verbs_tree.csv',encoding = 'utf-8-sig', index = False )
all_verbs

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion,Stress Number,First Character,Last Character,Last Character Number,Vowels Quantity
0,0,present,indicative,imperfective,non-causative,negative,1s,passive,intrans,a-me-*ne*-ger-i-em(ɛ),amneˈgerijemɛ,e,0.307692,0,ɛ,15,6
1,1,present,indicative,imperfective,non-causative,negative,2s,passive,intrans,a-me-*ne*-ger-i-i,amneˈgerii,e,0.400000,0,i,5,5
2,2,present,indicative,imperfective,non-causative,negative,3sm,passive,intrans,a-me-*ne*-ger-i-e,amneˈgerije,e,0.363636,0,e,3,5
3,3,present,indicative,imperfective,non-causative,negative,3sf,passive,intrans,a-me-*ne*-ger-i-iɛ,amneˈgeriijɛ,e,0.333333,0,ɛ,15,6
4,4,present,indicative,imperfective,non-causative,negative,1p,passive,intrans,a-me-*ne*-ger-i-om(ɛ),amneˈgerijomɛ,e,0.307692,0,ɛ,15,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44107,44107,past,subjunctive,pluperfect,non-causative,affirmative,3sm,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ,0.545455,0,ɛ,15,5
44108,44108,past,subjunctive,pluperfect,non-causative,affirmative,3sf,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ,0.545455,0,ɛ,15,5
44109,44109,past,subjunctive,pluperfect,non-causative,affirmative,1p,active,trans,ʌ-χʌst-*abimon*,ʌχʌstɛˈbimon,ɛ,0.500000,0,n,8,5
44110,44110,past,subjunctive,pluperfect,non-causative,affirmative,2p,active,trans,ʌ-χʌst-*abion*,ʌχʌstɛˈbijon,ɛ,0.500000,0,n,8,5
